# Day 5: A Maze of Twisty Trampolines, All Alike

An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

In [1]:
def step_inplace(jumps, ip):
    assert ip is not None
    new_ip = ip + jumps[ip]
    jumps[ip] += 1
    return new_ip if new_ip < len(jumps) else None


def how_many_steps(jumps, ip=0):
    jumps = list(jumps)
    steps = 0
    while ip is not None:
        ip = step_inplace(jumps, ip)
        steps += 1
    return steps

For example, consider the following list of jump offsets:

In [2]:
example = [0, 3, 0, 1, -3]

[0, 3, 0, 1, -3]

Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

In [3]:
# before we have taken any steps
jumps, ip = list(example), 0

# jump with offset 0 (that is, don't jump at all). fortunately, the instruction is then incremented to 1.
ip = step_inplace(jumps, ip)
assert jumps == [1, 3, 0, 1, -3] and ip == 0

# step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
ip = step_inplace(jumps, ip)
assert jumps == [2, 3, 0, 1, -3] and ip == 1

# jump all the way to the end; leave a 4 behind
ip = step_inplace(jumps, ip)
assert jumps == [2, 4, 0, 1, -3] and ip == 4

# go back to where we just were; increment -3 to -2
ip = step_inplace(jumps, ip)
assert jumps == [2, 4, 0, 1, -2] and ip == 1

# jump 4 steps forward, escaping the maze
ip = step_inplace(jumps, ip)
assert jumps == [2, 5, 0, 1, -2] and ip is None

In this example, the exit is reached in 5 steps.

In [4]:
assert how_many_steps(example) == 5

How many steps does it take to reach the exit?

In [5]:
puzzle = [int(line) for line in open('05.input').read().strip().splitlines()]
how_many_steps(puzzle)

374269

## Part 2

Now, the jumps are even stranger:
**after each jump, if the offset was three or more, instead decrease it by 1**.
Otherwise, increase it by 1 as before.

In [6]:
def step_inplace(jumps, ip):
    assert ip is not None
    offset = jumps[ip]
    new_ip = ip + offset
    jumps[ip] += -1 if offset >= 3 else 1
    return new_ip if new_ip < len(jumps) else None

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

In [7]:
assert how_many_steps(example) == 10

How many steps does it now take to reach the exit?

In [8]:
%%time
how_many_steps(puzzle)

CPU times: user 14.3 s, sys: 79.9 ms, total: 14.3 s
Wall time: 14.6 s


27720699

Just for fun, let's see how fast [numba](https://numba.pydata.org) is...

In [9]:
from numba import jit
step_inplace = jit(step_inplace)
how_many_steps = jit(how_many_steps)

CPUDispatcher(<function how_many_steps at 0x10f0a06a8>)

In [10]:
%%time
how_many_steps(puzzle)

CPU times: user 430 ms, sys: 10.9 ms, total: 441 ms
Wall time: 445 ms


27720699